In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import matplotlib.pyplot as plt

import scipy.signal as signal
import imageio as io
from tqdm import tqdm
import cv2
import shutil

import sys
sys.path.insert(1, '../numphly')
import numphly as nly



In [12]:
exp_target = 'R72'
dirloc = '../../../../../../Documents/'
output_fileloc = '../../FlyBehaviorSet/'
converted_folder = 'Converted/R72/'
# dirloc = '../../../../../../Desktop/FlyVideos/'
fileloc_set   = np.load('experiments_directory.npy')
num_file = len(fileloc_set)
videofile_set = np.load('experiments_well_setup.npy')
num_well = len(videofile_set)
experiments_index = np.load('experiments_index.npy')

experiments_mscore = np.load('experiments_mscore.npy',allow_pickle='TRUE').item()

crop_center = np.load('crop_h.npy').astype(int)


In [17]:
clip_folder = 'clips_for_labeling'
PATH = output_fileloc + clip_folder
if not os.path.exists(PATH):os.makedirs(PATH) 
else:print(PATH, 'exists')

DLC_label_folder= 'DLC_label_ALL'
PATH = output_fileloc + DLC_label_folder
if not os.path.exists(PATH):os.makedirs(PATH)
else:print(PATH, 'exists')

# OG_recording_folder= 'original_recording_for_tracking/R72'  

# PATH = output_fileloc + OG_recording_folder
# if not os.path.exists(PATH):os.makedirs(PATH)
# else:print(PATH, 'exists')
    
cropped_recording_folder= 'cropped_recording_for_tracking'  
PATH = output_fileloc + cropped_recording_folder
if not os.path.exists(PATH):os.makedirs(PATH)
else:print(PATH, 'exists')
    
    
    


../../FlyBehaviorSet/clips_for_labeling exists
../../FlyBehaviorSet/DLC_label_ALL exists
../../FlyBehaviorSet/cropped_recording_for_tracking exists


In [14]:
frame_rate = 60

height = 576 
width = 576
w_o = 576
h_o = w_o
gender = ['F','M']
long_folder= 'long'
PATH = output_fileloc + DLC_label_folder+'/'+long_folder
if not os.path.exists(PATH):os.makedirs(PATH)
else:print(PATH, 'exists')

    
    
short_folder= 'short/'+exp_target
PATH = output_fileloc + DLC_label_folder+'/'+short_folder
if not os.path.exists(PATH):os.makedirs(PATH)
else:print(PATH, 'exists')


buffer_1 = 15
buffer_0 = 2

../../FlyBehaviorSet/DLC_label_ALL/long exists
../../FlyBehaviorSet/DLC_label_ALL/short/R72 exists


# Crop Videos

In [5]:
# cropped_recordings = []
# PATH_FOI = output_fileloc + cropped_recording_folder
# w_o = 480
# h_o = w_o
# label = False
# for i in range(num_file):
# # for i in range(1):
#     experiment = experiments_mscore[fileloc_set[i]]
#     sexes = experiment['sex']
#     gtypes = experiment['genotype']
    
#     if "events" in experiment:
#         events_type  = experiment['events type']
#         events_loc   = experiment['events loc']
#         events       = experiment['events']
#         arena_circle = experiment['arena circle']
#         experiment_index = experiments_index[i]
#         print(fileloc_set[i],len(events_loc))
#         for j in range(num_well):
# #         for j in range(1):
#             well_arena = arena_circle[j]
#             event_type = events_type[j]
#             event_loc = events_loc[j]
#             well = videofile_set[j]
#             sex = sexes[j]
#             gtype = gtypes[j]
#             output_file_prefix = 'v_'+experiment_index+'_'+well
#             if sex != -1:
#                 event_ts = [63000]
#                 n_event = len(event_ts)
#                 circle = np.mean(well_arena, axis = 0)
#                 circle = np.round(circle).astype("int")
#                 event_ts_loc_m = np.zeros((1,2)).astype(int)
#                 event_ts_loc_m[0,0] = circle[0]
#                 event_ts_loc_m[0,1] = circle[1]
# #                 event_ts_loc_m = (height/2*np.ones((n_event,2))).astype(int)
#                 video   = output_fileloc + OG_recording_folder +'/'+output_file_prefix+'.mp4'
#                 print(video, output_file_prefix)

#                 video_list = nly.clip_videos(video, PATH_FOI, output_file_prefix, 
#                                 event_ts, event_ts_loc_m, 10, 10, int(frame_rate), h_o, w_o, label)
#                 k = video_list[0]
#                 newname = output_file_prefix+'_cropped.mp4'
#                 os.rename(PATH_FOI+'/'+k, PATH_FOI+'/'+newname)
#                 cropped_recordings.append(newname)

# np.save('cropped_recordings.npy', cropped_recordings)
# print(len(test_video_clips),event_ts_loc_m)

In [19]:
print(num_file)

15


In [16]:
label = True
w_o = 576
h_o = w_o
short_video_clips = []
for i in range(num_file):
# for i in [9]:
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    
    if "events" in experiment:
        events_type = experiment['events type']
        events_loc  = experiment['events loc']
        events = experiment['events']
        experiment_index = experiments_index[i]
        print(fileloc_set[i],len(events_loc))
        for j in range(num_well):
#         for j in [3]:
            event_ts = events[j]
            event_type = events_type[j]
            event_loc = np.ones(len(event_ts))*5
            n_event = len(event_ts)
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
            if sex != -1:
                event_ts_loc_m = (height/2*np.ones((n_event,2))).astype(int)
                # clipping videos for labeling and move them into a single directory
#                 video   = output_fileloc + OG_recording_folder +'/'+output_file_prefix+'.mp4'
                video   = output_fileloc+converted_folder+fileloc_set[i]+'well_'+videofile_set[j]+'_ts.mp4'
                print(output_file_prefix, sex, video)
                if n_event>0:
                    buffer_0 = 30
                    buffer_1 = 30
                    PATH_FOI = output_fileloc + DLC_label_folder+'/'+short_folder
                    print(event_ts.shape, event_ts_loc_m.shape)
                    video_list = nly.clip_videos(video, PATH_FOI, output_file_prefix, 
                                    event_ts, event_ts_loc_m, buffer_0, buffer_1, int(frame_rate), 
                                                 h_o, w_o, label, '.mp4')
                    
                    video_list = nly.clip_videos_name(video, PATH_FOI, output_file_prefix, 
                                    event_ts, event_ts_loc_m, buffer_0, buffer_1, int(frame_rate), 
                                                 h_o, w_o, label, '.mp4')
                    jj = 0
                    for k in video_list:
                        newname = 'v_'+format(sex,'01d')+'_'+format(gtype,'01d')+'_'+'type'+format(int(event_type[jj]),'01d')+'_'+'loc'+format(int(event_loc[jj]),'01d')+'_'+k[2:len(k)]
                        os.rename(PATH_FOI+'/'+k, PATH_FOI+'/'+newname)
                        short_video_clips.append(newname)
#                         if event_type[jj] == 3 and event_loc[jj]==0:
#                             print(newname)
                        jj = jj+1
# np.save('short_video_clips.npy', short_video_clips)

5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/ 8
v_05_08_24_exp1_A1 0 ../../FlyBehaviorSet/Converted/R72/5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/well_A1_ts.mp4
(21,) (21, 2)
v_05_08_24_exp1_B1 1 ../../FlyBehaviorSet/Converted/R72/5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/well_B1_ts.mp4
(3,) (3, 2)
v_05_08_24_exp1_D1 1 ../../FlyBehaviorSet/Converted/R72/5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/well_D1_ts.mp4
(17,) (17, 2)
v_05_08_24_exp1_A2 0 ../../FlyBehaviorSet/Converted/R72/5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/well_A2_ts.mp4
(5,) (5, 2)
v_05_08_24_exp1_B2 0 ../../FlyBehaviorSet/Converted/R72/5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/well_B2_ts.mp4
(10,) (10, 2)
5_8_24/5_8_24_R72A10TrpA1Expt2_video_20240508_142937_318/ 8
v_05_08_24_exp2_C1 0 ../../FlyBehaviorSet/Converted/R72/5_8_24/5_8_24_R72A10TrpA1Expt2_video_20240508_142937_318/well_C1_ts.mp4
(73,) (73, 2)
v_05_08_24_exp2_D1 0 ../../FlyBe

v_07_23_24_exp2_C1 1 ../../FlyBehaviorSet/Converted/R72/7_23_24/7_23_24_expt2_video_20240723_130206_295/well_C1_ts.mp4
(9,) (9, 2)
v_07_23_24_exp2_D1 1 ../../FlyBehaviorSet/Converted/R72/7_23_24/7_23_24_expt2_video_20240723_130206_295/well_D1_ts.mp4
(30,) (30, 2)
v_07_23_24_exp2_A2 0 ../../FlyBehaviorSet/Converted/R72/7_23_24/7_23_24_expt2_video_20240723_130206_295/well_A2_ts.mp4
(30,) (30, 2)
v_07_23_24_exp2_B2 0 ../../FlyBehaviorSet/Converted/R72/7_23_24/7_23_24_expt2_video_20240723_130206_295/well_B2_ts.mp4
(10,) (10, 2)
v_07_23_24_exp2_C2 0 ../../FlyBehaviorSet/Converted/R72/7_23_24/7_23_24_expt2_video_20240723_130206_295/well_C2_ts.mp4
(70,) (70, 2)
7_24_24/7_24_24_extp1_video_20240724_102123_758/ 8
v_07_24_24_exp1_A1 1 ../../FlyBehaviorSet/Converted/R72/7_24_24/7_24_24_extp1_video_20240724_102123_758/well_A1_ts.mp4
(41,) (41, 2)
v_07_24_24_exp1_B1 0 ../../FlyBehaviorSet/Converted/R72/7_24_24/7_24_24_extp1_video_20240724_102123_758/well_B1_ts.mp4
(6,) (6, 2)
v_07_24_24_exp1_C1 0 .

# Partial Video for testing

In [ ]:
label = True
w_o = 576
h_o = w_o

for i in range(num_file):
# for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    
    if "events" in experiment:
        events_type = experiment['events type']
        events_loc  = experiment['events loc']
        events = experiment['events']
        experiment_index = experiments_index[i]
        print(fileloc_set[i],len(events_loc))
        for j in range(num_well):
#         for j in range(1):
            event_ts = events[j]
            event_type = events_type[j]
            event_loc = events_loc[j]
            n_event = len(event_ts)
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
            if sex != -1:
                event_ts_loc_m = (height/2*np.ones((n_event,2))).astype(int)
                # clipping videos for labeling and move them into a single directory
                video   = output_fileloc + OG_recording_folder +'/'+output_file_prefix+'.mp4'
                if n_event>0:
                    print(video)
                    PATH_FOI = output_fileloc + DLC_label_folder+'/'+short_folder
#                     print(event_ts.shape, event_ts_loc_m.shape)
                    video_list = nly.clip_videos(video, PATH_FOI, output_file_prefix, 
                                    event_ts, event_ts_loc_m, buffer_0, buffer_1, int(frame_rate), 
                                                 h_o, w_o, label, '.avi')
                    
                    jj = 0
                    for k in video_list:
                        newname = 'v_'+format(sex,'01d')+'_'+format(gtype,'01d')+'_'+'type'+format(int(event_type[jj]),'01d')+'_'+'loc'+format(int(event_loc[jj]),'01d')+'_'+k[2:len(k)]
                        os.rename(PATH_FOI+'/'+k, PATH_FOI+'/'+newname)
                        short_video_clips.append(newname)
                        print(newname)
                        jj = jj+1


# Video clip for review

In [6]:
label = True
w_o = 576
h_o = w_o
long_video_clips = []
for i in range(len(fileloc_set)):
# for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    
    if "events" in experiment:
        events_type = experiment['events type']
        events_loc  = experiment['events loc']
        events = experiment['events']
        experiment_index = experiments_index[i]
        print(fileloc_set[i],len(events_loc))
        for j in range(num_well):
#         for j in range(1):
            event_ts = events[j]
            event_type = events_type[j]
            event_loc = events_loc[j]
            n_event = len(event_ts)
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
#             print(sex,gtype,len(event_type),len(event_loc),len(event_ts))
#             if sex == 100:
            if sex != -1:
                event_ts_loc_m = (height/2*np.ones((n_event,2))).astype(int)
                # clipping videos for labeling and move them into a single directory
#                 video   = output_fileloc + OG_recording_folder +'/'+output_file_prefix+'.mp4'
                video   = output_fileloc + clip_folder +'/'+output_file_prefix+'_ts.mp4'

                if n_event>0:
                    print(video, output_file_prefix)

                    PATH_FOI = output_fileloc + 'DLC_label'+'/'+long_folder
                    video_list = nly.clip_videos(video, PATH_FOI, output_file_prefix, 
                                    event_ts, event_ts_loc_m, buffer_1, buffer_1, 
                                                 int(frame_rate), h_o, w_o, label, '.mp4')
#                     jj = 0
#                     for k in video_list:
#                         newname = 'v_'+format(sex,'01d')+'_'+format(gtype,'01d')+'_'+'type'+format(int(event_type[jj]),'01d')+'_'+'loc'+format(int(event_loc[jj]),'01d')+'_'+k[2:len(k)]
#                         os.rename(PATH_FOI+'/'+k, PATH_FOI+'/'+newname)
#                         long_video_clips.append(newname)
#                         jj = jj+1
# np.save('long_video_clips.npy', long_video_clips)

5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/ 8
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp1_A1_ts.mp4 v_05_08_24_exp1_A1
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp1_B1_ts.mp4 v_05_08_24_exp1_B1
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp1_D1_ts.mp4 v_05_08_24_exp1_D1
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp1_A2_ts.mp4 v_05_08_24_exp1_A2
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp1_B2_ts.mp4 v_05_08_24_exp1_B2
5_8_24/5_8_24_R72A10TrpA1Expt2_video_20240508_142937_318/ 8
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp2_C1_ts.mp4 v_05_08_24_exp2_C1
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp2_D1_ts.mp4 v_05_08_24_exp2_D1
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp2_A2_ts.mp4 v_05_08_24_exp2_A2
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp2_B2_ts.mp4 v_05_08_24_exp2_B2
../../FlyBehaviorSet/clips_for_labeling/v_05_08_24_exp2_D2_ts.mp4 v_05_08_24_exp2_D2
5_9_24/5_9_24_r72A10_expt1_vid

In [7]:
label = False
long_video_clips = []
for i in range(len(fileloc_set)):
# for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    
    if "events" in experiment:
        events_type = experiment['events type']
        events_loc  = experiment['events loc']
        events = experiment['events']
        experiment_index = experiments_index[i]
        print(fileloc_set[i],len(events_loc))
        for j in range(num_well):
#         for j in range(1):
            event_ts = events[j]
            event_type = events_type[j]
            event_loc = events_loc[j]
            n_event = len(event_ts)
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
#             print(sex,gtype,len(event_type),len(event_loc),len(event_ts))
#             if sex == 100:
            if sex != -1:
                event_ts_loc_m = (height/2*np.ones((n_event,2))).astype(int)
                # clipping videos for labeling and move them into a single directory
                video   = output_fileloc + OG_recording_folder +'/'+output_file_prefix+'.mp4'
                
#                 cap = cv2.VideoCapture(video)
#                 h = int(cap.get(cv2.CAP_PROP_FOURCC))
#                 codec = chr(h&0xff) + chr((h>>8)&0xff) + chr((h>>16)&0xff) + chr((h>>24)&0xff)
#                 print(codec)
                if n_event>0:
                    print(video, output_file_prefix)

                    PATH_FOI = output_fileloc + DLC_label_folder+'/'+long_folder
                    video_list = nly.clip_videos(video, PATH_FOI, output_file_prefix, 
                                    event_ts, event_ts_loc_m, buffer_1, buffer_1, int(frame_rate), h_o, w_o, label)
                    jj = 0
                    for k in video_list:
                        
                        newname = 'v_'+format(sex,'01d')+'_'+format(gtype,'01d')+'_'+'type'+format(int(event_type[jj]),'01d')+'_'+'loc'+format(int(event_loc[jj]),'01d')+'_'+k[2:len(k)]
                        os.rename(PATH_FOI+'/'+k, PATH_FOI+'/'+newname)
                        long_video_clips.append(newname)
                        jj = jj+1
np.save('long_video_clips.npy', long_video_clips)

5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/ 8
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp1_A1.mp4 v_05_08_24_exp1_A1
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp1_B1.mp4 v_05_08_24_exp1_B1
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp1_D1.mp4 v_05_08_24_exp1_D1
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp1_A2.mp4 v_05_08_24_exp1_A2
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp1_B2.mp4 v_05_08_24_exp1_B2
5_8_24/5_8_24_R72A10TrpA1Expt2_video_20240508_142937_318/ 8
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp2_C1.mp4 v_05_08_24_exp2_C1
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp2_D1.mp4 v_05_08_24_exp2_D1
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp2_A2.mp4 v_05_08_24_exp2_A2
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp2_B2.mp4 v_05_08_24_exp2_B2
../../FlyBehaviorSet/orig

../../FlyBehaviorSet/original_recording_for_tracking/v_07_24_24_exp4_C2.mp4 v_07_24_24_exp4_C2


In [15]:

# np.save('short_video_clips.npy', short_video_clips)

In [ ]:
short_video_clips = np.load('short_video_clips.npy')

In [3]:
from os import listdir
from os.path import isfile, join
# mypath = '../../FlyBehaviorSet/DLC_models/floor_topview_12label_1unique-ziying-2025-01-04/videos'
mypath = '../../FlyBehaviorSet/DLC_models/wall_sideview_7labels-ziying-2025-01-01/videos'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
np.save('mp4filesname.npy',onlyfiles)
mp4filesname = onlyfiles
i = 0
for mp4file in mp4filesname:
    avifile = '../../FlyBehaviorSet/DLC_label_ALL/long/'+mp4file[0:len(mp4file)-3]+'avi'
    
#     print(avifile)
    my_file = Path(avifile)
    i = i+1
    print(i)
    if my_file.is_file():
        print(avifile)
        shutil.copyfile(avifile, '/home/ziyingc/Documents/videos/'+mp4file[0:len(mp4file)-3]+'avi')
    else:
        print('not yet')
        
            

1
../../FlyBehaviorSet/DLC_label_ALL/long/v_1_2_type3_loc1_05_09_24_exp2_D1_0027_113397.avi
2
../../FlyBehaviorSet/DLC_label_ALL/long/v_0_1_type1_loc1_05_08_24_exp1_B2_0001_017141.avi
3
../../FlyBehaviorSet/DLC_label_ALL/long/v_0_0_type1_loc1_05_09_24_exp1_B2_0016_044833.avi
4
../../FlyBehaviorSet/DLC_label_ALL/long/v_0_0_type1_loc1_05_08_24_exp2_D2_0002_029738.avi
5
../../FlyBehaviorSet/DLC_label_ALL/long/v_1_0_type2_loc1_05_13_24_exp1_A1_0010_017842.avi
6
../../FlyBehaviorSet/DLC_label_ALL/long/v_1_0_type2_loc1_05_13_24_exp1_A1_0015_021448.avi
7
../../FlyBehaviorSet/DLC_label_ALL/long/v_1_0_type3_loc1_05_13_24_exp2_C2_0004_012926.avi
8
../../FlyBehaviorSet/DLC_label_ALL/long/v_0_0_type1_loc1_05_08_24_exp2_D2_0015_062809.avi
9
../../FlyBehaviorSet/DLC_label_ALL/long/v_1_2_type3_loc1_05_17_24_exp1_B1_0032_061903.avi
10
../../FlyBehaviorSet/DLC_label_ALL/long/v_1_2_type3_loc1_05_17_24_exp1_B1_0041_093454.avi
11
../../FlyBehaviorSet/DLC_label_ALL/long/v_0_0_type1_loc1_05_08_24_exp2_D2_00

In [5]:
frames_for_labels = []
h_o = 480
w
# for i in range(len(fileloc_set)):
for i in range(1):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    arena_circle = experiment['arena circle']
    
    if "events" in experiment:
        events_type = experiment['events type']
        events_loc  = experiment['events loc']
        events = experiment['events']
        experiment_index = experiments_index[i]
        print(fileloc_set[i],len(events_loc))
#         for j in range(num_well):
        for j in range(1):
            event_ts = events[j]
            event_type = events_type[j]
            event_loc = events_loc[j]
            well_arena = arena_circle[j]
            n_event = len(event_ts)
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
#             print(sex,gtype,len(event_type),len(event_loc),len(event_ts))
#             if sex == 100:
            if sex != -1:
                if n_event>0:
                    circle = np.mean(well_arena, axis = 0)
                    circle = np.round(circle).astype("int")
                    event_ts_loc_m = np.zeros((1,2)).astype(int)
                    event_ts_loc_m[0,0] = circle[0]
                    event_ts_loc_m[0,1] = circle[1]
                    
                    

                    PATH_FOI = output_fileloc + OG_recording_folder +'/'+output_file_prefix
                    
                    if not os.path.exists(PATH):os.makedirs(PATH_FOI)
                    else:print(PATH_FOI, 'exists')
                    video   = PATH_FOI+'.mp4'
                    print(video, output_file_prefix)
                    video_list = nly.extract_frames(video, PATH_FOI, output_file_prefix, 
                                    event_ts, event_ts_loc_m, buffer_1, buffer_1, int(frame_rate), h_o, w_o)
                    jj = 0
                    for k in video_list:
                        frames_for_labels.append(k)
                        jj = jj+1


5_8_24/5_8_24_R72A10TrpA1Expt1_video_20240508_130433_636/ 8
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp1_A1 exists
../../FlyBehaviorSet/original_recording_for_tracking/v_05_08_24_exp1_A1.mp4 v_05_08_24_exp1_A1


TypeError: extract_frames() missing 1 required positional argument: 'label'

In [72]:

# for i in range(num_file):
for i in range(2):
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    
    if "events" in experiment:
        events_type = experiment['events type']
        events_loc  = experiment['events loc']
        events = experiment['events']
        experiment_index = experiments_index[i]
        for j in range(num_well):
#         for j in range(1):
            event_ts = events[j]
            event_type = events_type[j]
            event_loc = events_loc[j]
            n_event = len(event_ts)
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
            if sex != -1:
                event_ts_loc_m = (height/2*np.ones((n_event,2))).astype(int)
                # clipping videos for labeling and move them into a single directory
                video   = output_fileloc + OG_recording_folder +'/'+output_file_prefix+'.mp4'
                print(output_file_prefix, sex)
#                 if n_event>0:
#                     idx_time = event_ts >= 108000
#                     if sex == 0:
#                         idx_type = event_type == 1
#                         idx = np.logical_and(idx_time, idx_type)
#                         print('female HB:')
#                         if np.sum(idx) == 0:
#                             print(event_ts[idx]-108000)
#                         else:
#                             event_ts_chatgpt = event_ts[idx]-108000
#                             for k in event_ts_chatgpt:
#                                 print(k)
#                         print('')
#                     if sex == 1:
#                         idx_type = event_type == 2
#                         idx = np.logical_and(idx_time, idx_type)
#                         print('male UWE:')
#                         if np.sum(idx) == 0:
#                             print(event_ts[idx]-108000)
#                         else:
#                             event_ts_chatgpt = event_ts[idx]-108000
#                             for k in event_ts_chatgpt:
#                                 print(k)
                                
#                         idx_type = event_type == 3
#                         idx = np.logical_and(idx_time, idx_type)
#                         print('male L:')
#                         if np.sum(idx) == 0:
#                             print(event_ts[idx]-108000)
#                         else:
#                             event_ts_chatgpt = event_ts[idx]-108000
#                             for k in event_ts_chatgpt:
#                                 print(k)

v_05_08_24_exp1_A1 0
v_05_08_24_exp1_B1 1
v_05_08_24_exp1_D1 1
v_05_08_24_exp1_A2 0
v_05_08_24_exp1_B2 0
v_05_08_24_exp2_C1 0
v_05_08_24_exp2_D1 0
v_05_08_24_exp2_A2 1
v_05_08_24_exp2_B2 1
v_05_08_24_exp2_D2 0


In [76]:
# for i in range(num_file):
# for i in range(4,5):
for i in [2]:
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    
    if "events" in experiment:
        events_type = experiment['events type']
        events_loc  = experiment['events loc']
        events = experiment['events']
        experiment_index = experiments_index[i]
#         for j in range(num_well):
#         for j in range(1):
        for j in [5]:
            event_ts = events[j]
            event_type = events_type[j]
            event_loc = events_loc[j]
            n_event = len(event_ts)
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
            if sex != -1:
                event_ts_loc_m = (height/2*np.ones((n_event,2))).astype(int)
                # clipping videos for labeling and move them into a single directory
                video   = output_fileloc + OG_recording_folder +'/'+output_file_prefix+'.mp4'
                print(output_file_prefix, sex)
                
#                 # 05092024 exp1 well_B1_frames_123212-123280.mp4
#                 start = 123212
#                 end   = 123280
                
                # 05092024 exp1 well_A2_frames_067105-067211.mp4
                start = 67105
                end   = 67211
                idx_time = np.logical_and(event_ts >= start, event_ts <= end)
                print(event_ts)
#                 if sex == 0:
#                     idx_type = event_type == 1
#                     idx = np.logical_and(idx_time, idx_type)
#                     print('female HB:')
#                     if np.sum(idx) == 0:
#                         print(event_ts[idx])
#                     else:
#                         event_ts_chatgpt = event_ts[idx]
#                         for k in event_ts_chatgpt:
#                             print(k)
#                     print('')
#                 if sex == 1:
#                     idx_type = event_type == 2
#                     idx = np.logical_and(idx_time, idx_type)
#                     print('male UWE:')
#                     if np.sum(idx) == 0:
#                         print(event_ts[idx])
#                     else:
#                         event_ts_chatgpt = event_ts[idx]
#                         for k in event_ts_chatgpt:
#                             print(k)

#                     idx_type = event_type == 3
#                     idx = np.logical_and(idx_time, idx_type)
#                     print('male L:')
#                     if np.sum(idx) == 0:
#                         print(event_ts[idx])
#                     else:
#                         event_ts_chatgpt = event_ts[idx]
#                         for k in event_ts_chatgpt:
#                             print(k)

v_05_09_24_exp1_B2 0
[  5096  13241  14540  14560  14588  18606  27848  29719  29766  31412
  33746  33763  41690  41853  44265  44579  44833  45350  55566  55584
  73835  81368  82084  83548  83628 111181 121912]


In [63]:
# for i in range(num_file):
# for i in range(4,5):
for i in [3]:
    experiment = experiments_mscore[fileloc_set[i]]
    sexes = experiment['sex']
    gtypes = experiment['genotype']
    
    if "events" in experiment:
        events_type = experiment['events type']
        events_loc  = experiment['events loc']
        events = experiment['events']
        experiment_index = experiments_index[i]
        for j in range(num_well):
#         for j in range(1):
#         for j in [1]:
            event_ts = events[j]
            event_type = events_type[j]
            event_loc = events_loc[j]
            n_event = len(event_ts)
            well = videofile_set[j]
            sex = sexes[j]
            gtype = gtypes[j]
            output_file_prefix = 'v_'+experiment_index+'_'+well
            if sex != -1:
                event_ts_loc_m = (height/2*np.ones((n_event,2))).astype(int)
                # clipping videos for labeling and move them into a single directory
                video   = output_fileloc + OG_recording_folder +'/'+output_file_prefix+'.mp4'
                print(output_file_prefix, sex)

                start = 0
                end = 126000
                
                idx_time = np.logical_and(event_ts >= start, event_ts <= end)
#                 print(event_ts)
#                 print(event_type)
                if sex == 0:
                    idx_type = event_type == 1
                    idx = np.logical_and(idx_time, idx_type)
                    print('female HB:')
                    if np.sum(idx) == 0:
                        print(event_ts[idx])
                    else:
                        event_ts_chatgpt = event_ts[idx]
                        for k in event_ts_chatgpt:
                            print(k)
                    
                if sex == 1:
                    idx_type = event_type == 2
                    idx = np.logical_and(idx_time, idx_type)
                    print('male UWE:')
                    if np.sum(idx) == 0:
                        print(event_ts[idx])
                    else:
                        event_ts_chatgpt = event_ts[idx]
                        for k in event_ts_chatgpt:
                            print(k)

                    idx_type = event_type == 3
                    idx = np.logical_and(idx_time, idx_type)
                    print('male L:')
                    if np.sum(idx) == 0:
                        print(event_ts[idx])
                    else:
                        event_ts_chatgpt = event_ts[idx]
                        for k in event_ts_chatgpt:
                            print(k)

v_05_09_24_exp2_B1 0
female HB:
47026
47100
47162
47527
75282
76903
116159
116430
v_05_09_24_exp2_C1 1
male UWE:
[]
male L:
[]
v_05_09_24_exp2_D1 1
male UWE:
[]
male L:
65236
65889
70005
70315
70397
72503
72648
73073
73096
73216
74370
74399
90672
90771
90828
91102
92195
92254
92275
95452
95880
96545
96593
97409
99441
102811
104834
113397
120085
120133
123216
123194
v_05_09_24_exp2_A2 0
female HB:
6950
7027
7300
8399
8578
9198
10596
11171
12887
12999
13670
13730
23561
23728
24083
26398
27198
27795
29242
29286
31994
32208
34054
34817
39833
39865
39941
44060
52822
56058
56095
56236
56258
56274
56604
64317
69443
116206
116243
